In [2]:
!pip install split_folders

In [19]:
import splitfolders
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Sequential
import pickle


In [4]:
img_wid = 64
img_hig = 64


In [5]:
splitfolders.ratio(r'rps-cv-images',
                   output = 'output',
                   seed = 1377,
                   ratio = (.8,.1,.1),
                   group_prefix = None
                   )


Copying files: 1718 files [00:19, 86.28 files/s] 


In [6]:
test_datagen =datagen = ImageDataGenerator(
    rescale=1.0/255,        # Normalize pixel values to [0, 1]
    rotation_range=40,      # Randomly rotate images by up to 40 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2, # Randomly shift images vertically by 20% of the height
    shear_range=0.2,        # Randomly apply shearing transformations
    zoom_range=0.2,         # Randomly zoom in or out
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest'     # Fill empty pixels with nearest pixel values
)
train_datagen = datagen = ImageDataGenerator(
    rescale=1.0/255,        # Normalize pixel values to [0, 1]
    rotation_range=40,      # Randomly rotate images by up to 40 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2, # Randomly shift images vertically by 20% of the height
    shear_range=0.2,        # Randomly apply shearing transformations
    zoom_range=0.2,         # Randomly zoom in or out
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest'     # Fill empty pixels with nearest pixel values
)

In [9]:
train_datagen = train_datagen.flow_from_directory(directory = 'output/train',#location should be precise
                                                  target_size = (img_wid,img_hig),
                                                  batch_size = 128,
                                                  class_mode = 'categorical',
                                                  )

Found 1373 images belonging to 3 classes.


In [12]:
validation_data_generator = test_datagen.flow_from_directory(directory = 'output/test',
                                                target_size = (img_wid,img_hig),
                                                batch_size = 128,
                                                class_mode = 'categorical'
                                                )

Found 174 images belonging to 3 classes.


In [13]:
test_validation = test_datagen.flow_from_directory(directory = 'output/val',
                                                target_size = (img_wid,img_hig),
                                                batch_size = 128,
                                                class_mode = 'categorical'
                                                )

Found 171 images belonging to 3 classes.


In [14]:
model = Sequential()

In [15]:
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_wid, img_hig,3)))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())  # Flatten feature maps into a 1D vector

# Add dense layers
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(3, activation="softmax"))  # Output layer



c:\Users\nikhi\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,625,539 (6.20 MB)

 Trainable params: 1,625,539 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(train_datagen, steps_per_epoch = len(train_datagen),
                              epochs = 200,
                              validation_data = validation_data_generator,
                              validation_steps = len(validation_data_generator))

c:\Users\nikhi\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.4077 - loss: 1.2952 - val_accuracy: 0.4368 - val_loss: 1.0241
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 438ms/step - accuracy: 0.4408 - loss: 1.0324 - val_accuracy: 0.4310 - val_loss: 1.0069
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 452ms/step - accuracy: 0.4674 - loss: 1.0091 - val_accuracy: 0.4310 - val_loss: 0.9931
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 444ms/step - accuracy: 0.5018 - loss: 0.9914 - val_accuracy: 0.6264 - val_loss: 0.9679
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 460ms/step - accuracy: 0.5454 - loss: 0.9632 - val_accuracy: 0.6379 - val_loss: 0.9197
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 470ms/step - accuracy: 0.5970 - loss: 0.9126 - val_accuracy: 0.7356 - val_loss: 0.8858
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 437ms/step - accuracy: 0.6667 - loss: 0.8610 - val_accuracy: 0.7414 - val_loss: 0.7800
Epoch 8/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 434ms/step - accuracy: 0.6974 - loss: 0.7882 - val_accura

In [20]:
import numpy as np
print(np.array(model.evaluate(train_datagen)).round())
print(np.array(model.evaluate(validation_data_generator)).round())
print(np.array(model.evaluate(test_validation)).round())

11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 479ms/step - accuracy: 0.9935 - loss: 0.0338
[0. 1.]
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.9704 - loss: 0.0454
[0. 1.]
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.9805 - loss: 0.0310
[0. 1.]


In [21]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [30]:
img_path = r'prediction/PXL_20250126_054100416-removebg-preview (1).png'
img = image.load_img(img_path, target_size = (img_wid, img_hig))
img_arr = image.img_to_array(img)
img_arr = np.expand_dims(img_arr, axis = 0)
img_arr = img_arr/255.0

In [31]:
prediction = model.predict(img_arr)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


In [32]:
print(prediction.round())

[[0. 0. 1.]]


saving of the m

In [36]:
model_pkl_file = 'Saved_Model/model.pkl'



In [37]:
with open(model_pkl_file, 'wb') as file:
  pickle.dump(model, file)

In [38]:
with open(model_pkl_file, 'rb') as file:
  model = pickle.load(file)


In [39]:
y_predict = model.predict(img_arr)
print(y_predict.round())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
[[0. 0. 1.]]
